In [43]:
import pyarrow as pa

In [44]:
from pygdf.dataframe import DataFrame

In [45]:
import pandas as pd
import json
from shapely.geometry import MultiPoint,mapping
import numpy as np

In [46]:
uber_dataset = pd.read_csv("node_server/uploads/uber-dataset")

In [47]:
uber_dataset.dtypes

sourceid                                      int64
dstid                                         int64
hod                                           int64
mean_travel_time                            float64
standard_deviation_travel_time              float64
geometric_mean_travel_time                  float64
geometric_standard_deviation_travel_time    float64
dtype: object

In [48]:
def getCentroid(l):
    return mapping(MultiPoint(l).centroid)['coordinates']

In [49]:
mappings = pd.read_json("data/1_censustracts.json")

### Creating mappings dataframe from the json file

In [50]:
df_mappings = pd.DataFrame(columns=["id","lat","long"])

for key,val in mappings.features.iteritems():
    id = val['properties']['MOVEMENT_ID']
    lat,long = getCentroid(val['geometry']['coordinates'][0][0])
#     address = val['properties']['DISPLAY_NAME']
    data = np.array([id,lat,long])
    temp = pd.Series(data,index = ["id","lat","long"])
    df_mappings = df_mappings.append(temp,ignore_index=True)
df_mappings['id'] = df_mappings['id'].apply(np.int64)
df_mappings['lat'] = df_mappings['lat'].apply(np.float32)
df_mappings['long'] = df_mappings['long'].apply(np.float32)
# df_mappings['address'] = df_mappings['address'].apply(str)

## Converting 

In [51]:
sources_uber_dataset = pd.merge(uber_dataset,df_mappings,how = 'left',left_on ='sourceid', right_on='id');
sources_uber_dataset.rename(columns={'lat':'source_lat', 'long':'source_long'},inplace=True)
del sources_uber_dataset['id']

dst_uber_dataset = pd.merge(sources_uber_dataset,df_mappings,how = 'left',left_on ='dstid', right_on='id');
dst_uber_dataset.rename(columns={'lat':'dst_lat', 'long':'dst_long'},inplace=True)
del dst_uber_dataset['id']

In [52]:
dst_uber_dataset.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,source_lat,source_long,dst_lat,dst_long
0,6,5,17,2917.18,790.28,2810.98,1.32,-121.948433,38.366653,-121.361717,38.554436
1,3,38,20,339.74,254.41,287.54,1.67,-121.660217,37.150475,-121.481194,37.205841
2,2,61,7,1443.36,374.62,1397.73,1.29,-122.219406,37.469585,-122.050941,37.561699
3,7,25,2,152.00,180.40,95.34,2.61,-121.996101,38.325848,-121.979637,38.343208
4,2,79,22,2320.41,324.85,2299.22,1.14,-122.219406,37.469585,-122.419426,37.790886


In [53]:
pa_df = pa.RecordBatch.from_pandas(dst_uber_dataset)

In [54]:
path = 'node_server/uploads/uber-dataset-v1'

In [55]:
path = path+".arrow"
file = open(path, 'wb')
writer = pa.ipc.RecordBatchStreamWriter(file, pa_df.schema)
writer.write_batch(pa_df)
writer.close()
file.close()

In [57]:
def readArrowToDF(source):
    reader = pa.RecordBatchStreamReader(source)
    pa_df = reader.read_all()
    return pa_df.to_pandas()

In [58]:
tempdf = readArrowToDF('node_server/uploads/uber-dataset-v1.arrow')
#converting to csv
tempdf.to_csv('uber-dataset-v1.csv',index=False)

### Some test groupby functions on the dataframe using pygdf

In [5]:
df2 = readArrowToDF("node_server/uploads/uber-dataset-v1.arrow")

In [6]:
g_df = DataFrame.from_pandas(df2)

In [7]:
g_df.groupby(by=['hod']).agg({'hod':['count','count']}).head()

/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')
/conda/envs/gdf/lib/python3.6/

   hod hod_count
0    0  381642.0
1    1  349338.0
2    2  304540.0
3    3  274982.0
4    4  273401.0

In [8]:
g_df.head().to_pandas()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,source_lat,source_long,dst_lat,dst_long
0,6,5,17,2917.18,790.28,2810.98,1.32,-121.948433,38.366653,-121.361717,38.554436
1,3,38,20,339.74,254.41,287.54,1.67,-121.660217,37.150475,-121.481194,37.205841
2,2,61,7,1443.36,374.62,1397.73,1.29,-122.219406,37.469585,-122.050941,37.561699
3,7,25,2,152.00,180.40,95.34,2.61,-121.996101,38.325848,-121.979637,38.343208
4,2,79,22,2320.41,324.85,2299.22,1.14,-122.219406,37.469585,-122.419426,37.790886


In [9]:
a = g_df['hod']+g_df['hod']

In [10]:
g_df.add_column('hod*2',a)

In [11]:
g_df.head().to_pandas()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,source_lat,source_long,dst_lat,dst_long,hod*2
0,6,5,17,2917.18,790.28,2810.98,1.32,-121.948433,38.366653,-121.361717,38.554436,34
1,3,38,20,339.74,254.41,287.54,1.67,-121.660217,37.150475,-121.481194,37.205841,40
2,2,61,7,1443.36,374.62,1397.73,1.29,-122.219406,37.469585,-122.050941,37.561699,14
3,7,25,2,152.00,180.40,95.34,2.61,-121.996101,38.325848,-121.979637,38.343208,4
4,2,79,22,2320.41,324.85,2299.22,1.14,-122.219406,37.469585,-122.419426,37.790886,44


In [60]:
temp = g_df.groupby(by=['hod']).agg({'mean_travel_time':['mean','sum','std']})

/conda/envs/gdf/lib/python3.6/site-packages/numba/cuda/compiler.py:236: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


In [42]:
temp

    hod mean_travel_time_mean mean_travel_time_sum mean_travel_time_std
 0    0    1182.6042355137783         451331445.65    636.6144405380031
 1    1    1157.3405136284935   404303020.34999996    640.6810597885919
 2    2    1078.3579586916835         328403132.74    601.9580064100818
 3    3    1066.3307426302958         293221760.27    602.9960825330071
 4    4     1072.287644266013         293164514.23    596.1968492434379
 5    5    1181.3500912801521         357328868.86     697.783428367376
 6    6     1416.764673345385         527771759.35    842.6703223999089
 7    7    1671.1551026953978    724063042.5000001    954.6924313225265
 8    8    1671.1828021911426    760211029.6199999    888.0069238408861
 9    9     1553.455466028905    731228575.8699999    815.1537911630599
10   10    1493.1323787893502         709391672.56    785.1126356461986
11   11    1480.4418659098724         694722513.09    780.1713432950849
12   12    1498.5335853383237         704177415.62    782.023823

In [23]:
temp_dict = {'mean_travel_time':['mean','sum','std']}

In [37]:
list(temp_dict.values())

[['mean', 'sum', 'std']]

In [40]:
a = ':'.join(list(temp_dict.keys())+list(temp_dict.values())[0])

'mean_travel_time:mean:sum:std'

In [66]:
temp.nlargest(10,)

TypeError: __getitem__ on type ['mean_travel_time_mean', 'mean_travel_time_sum'] is not supported